In [ ]:
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
import time
from datetime import datetime, timedelta
from IPython.display import clear_output, display

def get_eurcad_data(interval="5m", period="1d"):
    try:
        eurcad = yf.Ticker("EURCAD=X")
        data = eurcad.history(period=period, interval=interval)
        return data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def market_sentiment(data):
    return data['Close'].pct_change().rolling(window=10).mean()

def trend_riding(data):
    sma_short = data['Close'].rolling(window=9).mean()
    sma_long = data['Close'].rolling(window=18).mean()
    return sma_short.iloc[-1] > sma_long.iloc[-1]

def breakout_fading(data):
    recent_high = data['High'].rolling(window=10).max()
    recent_low = data['Low'].rolling(window=10).min()
    return (data['Close'].iloc[-1] > recent_high.iloc[-2]) or (data['Close'].iloc[-1] < recent_low.iloc[-2])

def breakout_trading(data):
    breakout_threshold = data['Close'].rolling(window=20).max()
    return data['Close'].iloc[-1] > breakout_threshold.iloc[-1]

def decreased_volatility_breakout(data):
    volatility = data['Close'].rolling(window=20).std()
    return volatility.iloc[-1] < volatility.mean()

def carry_trade():
    # Placeholder: Normally requires interest rate differentials
    return False

def news_straddling():
    # Placeholder: Requires news event detection
    return False

def analyze_and_generate_signals(data):
    if data is None or data.empty or len(data) < 20:
        return None, None
    
    last_price = data['Close'].iloc[-1]
    current_time = datetime.utcnow() - timedelta(hours=3)  # UTC -3
    trade_time = (current_time + timedelta(minutes=5)).strftime("%H:%M")
    
    signal_type = None
    
    if market_sentiment(data).iloc[-1] > 0:
        signal_type = "CALL 🟩"
    elif market_sentiment(data).iloc[-1] < 0:
        signal_type = "PUT 🟥"
    
    if trend_riding(data):
        signal_type = "CALL 🟩"
    
    if breakout_fading(data):
        signal_type = "PUT 🟥"
    
    if breakout_trading(data):
        signal_type = "CALL 🟩"
    
    if decreased_volatility_breakout(data):
        signal_type = "CALL 🟩"
    
    if carry_trade():
        signal_type = "CALL 🟩"
    
    if news_straddling():
        signal_type = "CALL 🟩"
    
    if signal_type:
        signal_text = f"💰 5 minutes expiry\nEUR/CAD;{trade_time};{signal_type}\n🕐 TIME TO {trade_time}"  
        return signal_text, trade_time
    return None, None

def create_candlestick_chart(data, title):
    if data is None or data.empty:
        print(f"No data to display for {title}.")
        return None
    
    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'])])
    fig.update_layout(title=title, xaxis_title='Time', yaxis_title='Price', xaxis_rangeslider_visible=False)
    return fig

def update_charts_and_signals():
    signals_history = []
    try:
        while True:
            data_5min = get_eurcad_data(interval="5m")
            fig_5min = create_candlestick_chart(data_5min, "EUR/CAD 5-Minute Chart")
            
            if fig_5min is not None:
                signal_text, trade_time = analyze_and_generate_signals(data_5min)
                
                if signal_text:
                    signals_history.append(signal_text)
                    signals_history = signals_history[-5:]
                    clear_output(wait=True)
                    display(fig_5min)
                    print("Trading Signals:")
                    for signal in signals_history:
                        print(signal)
                else:
                    clear_output(wait=True)
                    display(fig_5min)
                    print("No New Trading Signals")
            
            time.sleep(600)  # 10-minute interval
    except KeyboardInterrupt:
        print("Chart update stopped.")

if __name__ == "__main__":
    update_charts_and_signals()

Trading Signals:
💰 5 minutes expiry
EUR/CAD;04:37;CALL 🟩
🕐 TIME TO 04:37
💰 5 minutes expiry
EUR/CAD;04:47;CALL 🟩
🕐 TIME TO 04:47
💰 5 minutes expiry
EUR/CAD;04:57;CALL 🟩
🕐 TIME TO 04:57
💰 5 minutes expiry
EUR/CAD;05:07;CALL 🟩
🕐 TIME TO 05:07
💰 5 minutes expiry
EUR/CAD;05:17;PUT 🟥
🕐 TIME TO 05:17
